## Use Code Execution in Conversation


Writing and executing code is necessary for many tasks such as data analysis, machine learning, and mathematical modeling. In AutoGen, coding can be a conversation between a code writer agent and a code executor agent, mirroring the interaction between a programmer and a code interpreter.

In [ ]:
from autogen import config_list_from_json

myModelname="GPT35TUEBOFIRST" #your model Name

#Read config File AOAI_CONFIG_LIST.json
config_list = config_list_from_json(
    env_or_file="..\AOAI_CONFIG_LIST.json",
    filter_dict={"model": myModelname}
    )

# LLM config file for myModelname (your model Name)

llm_myModelName_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }

The code writer agent can be powered by an LLM such as GPT-4 with code-writing capability. And the code executor agent is powered by a code executor.

The following is an agent with a code writer role specified using system_message. The system message contains important instruction on how to use the code executor in the code executor agent.

In [ ]:
from autogen import ConversableAgent
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=llm_myModelName_config,
    code_execution_config=False,  # Turn off code execution for this agent. This agend write the code but it dosen't execute the code.!!!!!
)

In [ ]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

Play: 

During the previous chat session, human input was requested each time the code executor agent responded to ensure that the code was safe to execute.

if the code propousal is good just press enter. After the output is print, enter Exit.

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    #message="Write Python function  code to calculate the 14th Fibonacci number",
    message="Write Python function  code to calculate the 2 plus 2",
)

Now we can try a more complex example that involves querying the web. Let’s say we want to get the the stock price gains year-to-date for Tesla and Meta (formerly Facebook). We can also use the two agents with several iterations of conversation.

In [ ]:
%pip install yfinance
%pip install matplotlib


Play...

Accept the code propusal just click enter.
Aftyer the file is save, enter Exit.

In [ ]:
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=f"Today is {today}. Write Python function code to plot TSLA's and META's "
    "stock price gains YTD, and save the plot to a file named 'stock_gains.png'."
    " Don't create a phyton code file, execute the code in the phyton enviroment.",
)

Finally, shows the Graph

In [ ]:
import os
from IPython.display import Image
fileName=os.path.join(temp_dir.name, "stock_gains.png")
print(fileName)
Image(fileName)